## Libs

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
import math

import matplotlib.pyplot as plt


## Functions

In [ ]:
def eth_price_dai(df):
    return df['DAI_balance'] / df['ETH_balance']

def uni_price_dai(df):
    return 2 * df['DAI_balance'] / df['UNI_supply']

def gain_LP(df):
    return df['UNI_price_DAI'] / df['UNI_price_DAI'][0] - 1
    
def gain_ETH_hodler(df):
    return df['ETH_price_DAI'] / df['ETH_price_DAI'][0] - 1

def gain_50_50(df):
    a = 0.5 * df['ETH_price_DAI'][0] + 0.5 * df['ETH_price_DAI']
    a = a.values
    return a / a[0] - 1

def gain_LP_interest(df):
    return (df['UNI_price_DAI'] + df['DAI_Interest']) / df['UNI_price_DAI'][0] - 1

def gain_50_50_interest(df):
    a = (0.5 * df['ETH_price_DAI'][0] + df['DAI_Interest']) + (0.5 * df['ETH_price_DAI'])
    a = a.values
    return a / a[0] - 1

def calculate_dai_interest(df,rt):
    df['DAI_Interest'] = df['ETH_price_DAI'][0] * df['total_time']*rt
    return df

def calculate_delta_time(df,df_time):
    df = pd.concat([df,df_time],axis=1)
    df['delta_time'] = df['block_timestamp'].diff()
    df['delta_time'] = df['delta_time'].fillna(pd.Timedelta(seconds=0))
    df['delta_time'] = df['delta_time'].apply(lambda x: x.total_seconds())
    df['total_time'] = df['delta_time'].cumsum()
    return df

## Importing scenarios

In [ ]:
model_hist_v1 = pd.read_csv('output/model_current.csv',index_col=[0]).reset_index(drop=True)
model_hist_v2 = pd.read_csv('output/model_0-25%.csv',index_col=[0]).reset_index(drop=True)
model_hist_v3 = pd.read_csv('output/model_0-35%.csv',index_col=[0]).reset_index(drop=True)

model_fut_v1 = pd.read_csv('output/model_future_v1.csv',index_col=[0]).reset_index(drop=True)
model_fut_v2 = pd.read_csv('output/model_future_v2.csv',index_col=[0]).reset_index(drop=True)
model_fut_v3 = pd.read_csv('output/model_future_v3.csv',index_col=[0]).reset_index(drop=True)
model_fut_v4 = pd.read_csv('output/model_future_v4.csv',index_col=[0]).reset_index(drop=True)
model_fut_v5 = pd.read_csv('output/model_future_v5.csv',index_col=[0]).reset_index(drop=True)
model_fut_v6 = pd.read_csv('output/model_future_v6.csv',index_col=[0]).reset_index(drop=True)
model_fut_v7 = pd.read_csv('output/model_future_v7.csv',index_col=[0]).reset_index(drop=True)
model_fut_v8 = pd.read_csv('output/model_future_v8.csv',index_col=[0]).reset_index(drop=True)

# model_history.columns = ['model_DAI_balance', 'model_ETH_balance', 'model_UNI_supply']

## Renaming Columns

In [ ]:
model_hist_v1.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_hist_v2.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_hist_v3.columns = ['DAI_balance','ETH_balance','UNI_supply']

model_fut_v1.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_fut_v2.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_fut_v3.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_fut_v4.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_fut_v5.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_fut_v6.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_fut_v7.columns = ['DAI_balance','ETH_balance','UNI_supply']
model_fut_v8.columns = ['DAI_balance','ETH_balance','UNI_supply']

## Converting to Base Numbers

In [ ]:
model_hist_v1 = model_hist_v1.astype(float)
model_hist_v2 = model_hist_v2.astype(float)
model_hist_v3 = model_hist_v3.astype(float)

model_fut_v1 = model_fut_v1.astype(float)
model_fut_v2 = model_fut_v2.astype(float)
model_fut_v3 = model_fut_v3.astype(float)
model_fut_v4 = model_fut_v4.astype(float)
model_fut_v5 = model_fut_v5.astype(float)
model_fut_v6 = model_fut_v6.astype(float)
model_fut_v7 = model_fut_v7.astype(float)
model_fut_v8 = model_fut_v8.astype(float)

In [ ]:
model_hist_v1 = model_hist_v1*1e-18
model_hist_v2 = model_hist_v2*1e-18
model_hist_v3 = model_hist_v3*1e-18

model_fut_v1 = model_fut_v1*1e-18
model_fut_v2 = model_fut_v2*1e-18
model_fut_v3 = model_fut_v3*1e-18
model_fut_v4 = model_fut_v4*1e-18
model_fut_v5 = model_fut_v5*1e-18
model_fut_v6 = model_fut_v6*1e-18
model_fut_v7 = model_fut_v7*1e-18
model_fut_v8 = model_fut_v8*1e-18

## Adding Price Columns

In [ ]:
model_hist_v1['ETH_price_DAI'] = eth_price_dai(model_hist_v1)
model_hist_v1['UNI_price_DAI'] = uni_price_dai(model_hist_v1)

model_hist_v2['ETH_price_DAI'] = eth_price_dai(model_hist_v2)
model_hist_v2['UNI_price_DAI'] = uni_price_dai(model_hist_v2)

model_hist_v3['ETH_price_DAI'] = eth_price_dai(model_hist_v3)
model_hist_v3['UNI_price_DAI'] = uni_price_dai(model_hist_v3)

model_fut_v1['ETH_price_DAI'] = eth_price_dai(model_fut_v1)
model_fut_v1['UNI_price_DAI'] = uni_price_dai(model_fut_v1)

model_fut_v2['ETH_price_DAI'] = eth_price_dai(model_fut_v2)
model_fut_v2['UNI_price_DAI'] = uni_price_dai(model_fut_v2)

model_fut_v3['ETH_price_DAI'] = eth_price_dai(model_fut_v3)
model_fut_v3['UNI_price_DAI'] = uni_price_dai(model_fut_v3)

model_fut_v4['ETH_price_DAI'] = eth_price_dai(model_fut_v4)
model_fut_v4['UNI_price_DAI'] = uni_price_dai(model_fut_v4)

model_fut_v5['ETH_price_DAI'] = eth_price_dai(model_fut_v5)
model_fut_v5['UNI_price_DAI'] = uni_price_dai(model_fut_v5)

model_fut_v6['ETH_price_DAI'] = eth_price_dai(model_fut_v6)
model_fut_v6['UNI_price_DAI'] = uni_price_dai(model_fut_v6)

model_fut_v7['ETH_price_DAI'] = eth_price_dai(model_fut_v7)
model_fut_v7['UNI_price_DAI'] = uni_price_dai(model_fut_v7)

model_fut_v8['ETH_price_DAI'] = eth_price_dai(model_fut_v8)
model_fut_v8['UNI_price_DAI'] = uni_price_dai(model_fut_v8)

## Adding Gain Columns

In [ ]:
model_hist_v1['UNI_hodler_return'] = gain_LP(model_hist_v1)
model_hist_v1['ETH_hodler_return'] = gain_ETH_hodler(model_hist_v1)
model_hist_v1['50_50_hodler_return'] = gain_50_50(model_hist_v1)

model_hist_v2['UNI_hodler_return'] = gain_LP(model_hist_v2)
model_hist_v2['ETH_hodler_return'] = gain_ETH_hodler(model_hist_v2)
model_hist_v2['50_50_hodler_return'] = gain_50_50(model_hist_v2)

model_hist_v3['UNI_hodler_return'] = gain_LP(model_hist_v3)
model_hist_v3['ETH_hodler_return'] = gain_ETH_hodler(model_hist_v3)
model_hist_v3['50_50_hodler_return'] = gain_50_50(model_hist_v3)

model_fut_v1['UNI_hodler_return'] = gain_LP(model_fut_v1)
model_fut_v1['ETH_hodler_return'] = gain_ETH_hodler(model_fut_v1)
model_fut_v1['50_50_hodler_return'] = gain_50_50(model_fut_v1)

model_fut_v2['UNI_hodler_return'] = gain_LP(model_fut_v2)
model_fut_v2['ETH_hodler_return'] = gain_ETH_hodler(model_fut_v2)
model_fut_v2['50_50_hodler_return'] = gain_50_50(model_fut_v2)

model_fut_v3['UNI_hodler_return'] = gain_LP(model_fut_v3)
model_fut_v3['ETH_hodler_return'] = gain_ETH_hodler(model_fut_v3)
model_fut_v3['50_50_hodler_return'] = gain_50_50(model_fut_v3)

model_fut_v4['UNI_hodler_return'] = gain_LP(model_fut_v4)
model_fut_v4['ETH_hodler_return'] = gain_ETH_hodler(model_fut_v4)
model_fut_v4['50_50_hodler_return'] = gain_50_50(model_fut_v4)

model_fut_v5['UNI_hodler_return'] = gain_LP(model_fut_v5)
model_fut_v5['ETH_hodler_return'] = gain_ETH_hodler(model_fut_v5)
model_fut_v5['50_50_hodler_return'] = gain_50_50(model_fut_v5)

model_fut_v6['UNI_hodler_return'] = gain_LP(model_fut_v6)
model_fut_v6['ETH_hodler_return'] = gain_ETH_hodler(model_fut_v6)
model_fut_v6['50_50_hodler_return'] = gain_50_50(model_fut_v6)

model_fut_v7['UNI_hodler_return'] = gain_LP(model_fut_v7)
model_fut_v7['ETH_hodler_return'] = gain_ETH_hodler(model_fut_v7)
model_fut_v7['50_50_hodler_return'] = gain_50_50(model_fut_v7)

model_fut_v8['UNI_hodler_return'] = gain_LP(model_fut_v8)
model_fut_v8['ETH_hodler_return'] = gain_ETH_hodler(model_fut_v8)
model_fut_v8['50_50_hodler_return'] = gain_50_50(model_fut_v8)

## Adding Date Column

In [ ]:
uniswap_events = pd.read_pickle('model/parts/uniswap_events.pickle')
uniswap_events = uniswap_events.loc[2:,:][['block_timestamp']].reset_index(drop=True)

In [ ]:
model_hist_v1 = calculate_delta_time(model_hist_v1,uniswap_events)
model_hist_v2 = calculate_delta_time(model_hist_v2,uniswap_events)
model_hist_v3 = calculate_delta_time(model_hist_v3,uniswap_events)

In [ ]:
model_fut_v1['delta_time'] = 16.404
model_fut_v2['delta_time'] = 16.404
model_fut_v3['delta_time'] = 16.404
model_fut_v4['delta_time'] = 16.404
model_fut_v5['delta_time'] = 16.404
model_fut_v6['delta_time'] = 16.404
model_fut_v7['delta_time'] = 16.404
model_fut_v8['delta_time'] = 16.404

model_fut_v1['total_time'] = model_fut_v1['delta_time'].cumsum()
model_fut_v2['total_time'] = model_fut_v2['delta_time'].cumsum()
model_fut_v3['total_time'] = model_fut_v3['delta_time'].cumsum()
model_fut_v4['total_time'] = model_fut_v4['delta_time'].cumsum()
model_fut_v5['total_time'] = model_fut_v5['delta_time'].cumsum()
model_fut_v6['total_time'] = model_fut_v6['delta_time'].cumsum()
model_fut_v7['total_time'] = model_fut_v7['delta_time'].cumsum()
model_fut_v8['total_time'] = model_fut_v8['delta_time'].cumsum()

## Adding Interest Bearing Columns

In [ ]:
# https://defirate.com/dai/
rt_dai_year = 4.78*1e-2
rt_dai_second = rt_dai_year/(365*24*60*60)

In [ ]:
model_hist_v1 = calculate_dai_interest(model_hist_v1,rt_dai_second)
model_hist_v2 = calculate_dai_interest(model_hist_v2,rt_dai_second)
model_hist_v3 = calculate_dai_interest(model_hist_v3,rt_dai_second)

model_fut_v1 = calculate_dai_interest(model_fut_v1,rt_dai_second)
model_fut_v2 = calculate_dai_interest(model_fut_v2,rt_dai_second)
model_fut_v3 = calculate_dai_interest(model_fut_v3,rt_dai_second)
model_fut_v4 = calculate_dai_interest(model_fut_v4,rt_dai_second)
model_fut_v5 = calculate_dai_interest(model_fut_v5,rt_dai_second)
model_fut_v6 = calculate_dai_interest(model_fut_v6,rt_dai_second)
model_fut_v7 = calculate_dai_interest(model_fut_v7,rt_dai_second)
model_fut_v8 = calculate_dai_interest(model_fut_v8,rt_dai_second)

## Adding Gain with interest columns

In [ ]:
model_hist_v1['UNI_hodler_return_interest'] = gain_LP_interest(model_hist_v1)
model_hist_v1['50_50_hodler_return_interest'] = gain_50_50_interest(model_hist_v1)

model_hist_v2['UNI_hodler_return_interest'] = gain_LP_interest(model_hist_v2)
model_hist_v2['50_50_hodler_return_interest'] = gain_50_50_interest(model_hist_v2)

model_hist_v3['UNI_hodler_return_interest'] = gain_LP_interest(model_hist_v3)
model_hist_v3['50_50_hodler_return_interest'] = gain_50_50_interest(model_hist_v3)

model_fut_v1['UNI_hodler_return_interest'] = gain_LP_interest(model_fut_v1)
model_fut_v1['50_50_hodler_return_interest'] = gain_50_50_interest(model_fut_v1)

model_fut_v2['UNI_hodler_return_interest'] = gain_LP_interest(model_fut_v2)
model_fut_v2['50_50_hodler_return_interest'] = gain_50_50_interest(model_fut_v2)

model_fut_v3['UNI_hodler_return_interest'] = gain_LP_interest(model_fut_v3)
model_fut_v3['50_50_hodler_return_interest'] = gain_50_50_interest(model_fut_v3)

model_fut_v4['UNI_hodler_return_interest'] = gain_LP_interest(model_fut_v4)
model_fut_v4['50_50_hodler_return_interest'] = gain_50_50_interest(model_fut_v4)

model_fut_v5['UNI_hodler_return_interest'] = gain_LP_interest(model_fut_v5)
model_fut_v5['50_50_hodler_return_interest'] = gain_50_50_interest(model_fut_v5)

model_fut_v6['UNI_hodler_return_interest'] = gain_LP_interest(model_fut_v6)
model_fut_v6['50_50_hodler_return_interest'] = gain_50_50_interest(model_fut_v6)

model_fut_v7['UNI_hodler_return_interest'] = gain_LP_interest(model_fut_v7)
model_fut_v7['50_50_hodler_return_interest'] = gain_50_50_interest(model_fut_v7)

model_fut_v8['UNI_hodler_return_interest'] = gain_LP_interest(model_fut_v8)
model_fut_v8['50_50_hodler_return_interest'] = gain_50_50_interest(model_fut_v8)


## Generating return Graphs

In [ ]:
df_list = [model_hist_v1,model_hist_v2,model_hist_v3,model_fut_v1,model_fut_v2,model_fut_v3,model_fut_v4,model_fut_v5,model_fut_v6,model_fut_v7,model_fut_v8]

In [ ]:
for df in df_list:
    plt.figure(figsize=(15,5))
    ax = plt.subplot(121)
    ax.set_ylabel('ROI')
    df[['UNI_hodler_return','50_50_hodler_return','ETH_hodler_return']].plot(ax=ax)
    ax = plt.subplot(122)
    ax.set_ylabel('ROI')
    df[['UNI_hodler_return','UNI_hodler_return_interest']].plot(ax=ax)
    
    plt.figure(figsize=(15,5))
    ax = plt.subplot(121)
    ax.set_ylabel('ROI')
    df[['50_50_hodler_return','50_50_hodler_return_interest']].plot(ax=ax)
    ax = plt.subplot(122)
    ax.set_ylabel('ROI')
    df[['UNI_hodler_return_interest','50_50_hodler_return_interest','ETH_hodler_return']].plot(ax=ax)

## Generating Historical Comparisons

In [ ]:
df_h1 = model_hist_v1[['DAI_balance','ETH_balance','UNI_supply','UNI_hodler_return','UNI_hodler_return_interest']]
df_h2 = model_hist_v2[['DAI_balance','ETH_balance','UNI_supply','UNI_hodler_return','UNI_hodler_return_interest']]
df_h3 = model_hist_v3[['DAI_balance','ETH_balance','UNI_supply','UNI_hodler_return','UNI_hodler_return_interest']]

df_h1.columns = ['DAI_balance_0.30','ETH_balance_0.30','UNI_supply_0.30','UNI_hodler_return_0.30','UNI_hodler_return_interest_0.30']
df_h2.columns = ['DAI_balance_0.25','ETH_balance_0.25','UNI_supply_0.25','UNI_hodler_return_0.25','UNI_hodler_return_interest_0.25']
df_h3.columns = ['DAI_balance_0.35','ETH_balance_0.35','UNI_supply_0.35','UNI_hodler_return_0.35','UNI_hodler_return_interest_0.35']

df_hist_compare = pd.concat([df_h1,df_h2],axis=1)
df_hist_compare = pd.concat([df_hist_compare,df_h3],axis=1)

In [ ]:
df_hist_compare.head()

In [ ]:
plt.figure(figsize=(15,5))
ax = plt.subplot(111)
ax.set_title('DAI Liquidity')
ax.set_ylabel('DAI Amount')
df_hist_compare[['DAI_balance_0.30','DAI_balance_0.25','DAI_balance_0.35']].plot(ax=ax)

plt.figure(figsize=(15,5))
ax = plt.subplot(111)
ax.set_title('ETH Liquidity')
ax.set_ylabel('ETH Amount')
df_hist_compare[['ETH_balance_0.30','ETH_balance_0.25','ETH_balance_0.35']].plot(ax=ax)

plt.figure(figsize=(15,5))
ax = plt.subplot(111)
ax.set_title('UNI Supply')
ax.set_ylabel('UNI Amount')
df_hist_compare[['UNI_supply_0.30','UNI_supply_0.25','UNI_supply_0.35']].plot(ax=ax)

In [ ]:
plt.figure(figsize=(15,8))
ax = plt.subplot(111)
ax.set_title('LP return')
df_hist_compare[['UNI_hodler_return_0.30','UNI_hodler_return_0.25','UNI_hodler_return_0.35']].plot(ax=ax)


In [ ]:
plt.figure(figsize=(15,8))
ax = plt.subplot(111)
ax.set_title('LP return')
df_hist_compare[['UNI_hodler_return_interest_0.30','UNI_hodler_return_interest_0.25','UNI_hodler_return_0.35']].plot(ax=ax)
